# Plotting

This section introduces UXarray's plotting API, showcasing how to visualize both data variables and grid topology.

```{seealso}
For a more guided walkthrough of the unstructured grid visualization using UXarray, please refer to the [Unstructured Grid Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/README.html).
```

In [ ]:
import uxarray as ux

## Data

For most of the examples in this notebook, we will be using a simple mesh consisting of 4 hexagons, with sample data mapped to the faces, edges, and nodes.

In [ ]:
grid_path = "../../test/meshfiles/ugrid/quad-hexagon/grid.nc"

data_paths = [
    "../../test/meshfiles/ugrid/quad-hexagon/random-node-data.nc",
    "../../test/meshfiles/ugrid/quad-hexagon/random-edge-data.nc",
    "../../test/meshfiles/ugrid/quad-hexagon/random-face-data.nc",
]

uxds = ux.open_mfdataset(grid_path, data_paths)

uxds

## API Design

Below is a high-level overview of UXarray's plotting API. 

<img src="../_static/examples/plot/uxarray-plot-api-design.png" alt="UXarray Plotting API Design" width="800"/>

Key takeaways from this design are that:

* UXarray’s unified grid representation (through the UGRID conventions) means that all visualization functionality is agnostic to the grid format initially provided by the user.
* Each Uxarray data structure (i.e. `Grid`, `UxDataset`, `UxDataArray`) has its own `.plot` accessor, which is used to call plotting routines.
* The visualization functionality through these `.plot` accessors use `HoloViz` packages’ plotting functions, wrapping them in a way to exploit all the information that comes from unstructured grids (e.g. connectivity) and provide our unstructured grids-specific functions in the most convenient way for the user.

We can see what the default plotting methods do below:



In [ ]:
uxds.uxgrid.plot().opts(width=600, height=400, title="Default Grid Plotting Method")

In [ ]:
uxds["random_data_face"].plot().opts(
    width=600, height=400, cmap="inferno", title="Default UxDataArray Plotting Method"
)

Since UXarray's plotting API is written primarily using `HoloViews`, we can use either the Matplotlib or Bokeh backends for rendering plots. This can be done by setting the `backend` parameter to either `"matplotlib"` or `"bokeh"`

In [ ]:
uxds["random_data_face"].plot(backend="matplotlib").opts(
    fig_size=200, cmap="inferno", title="Matplotlib Backend"
)

## Visualizing  Grid Topology

No matter what analysis operations you are performing on the data, visualization of the geometric elements of an unstructured grid (i.e. nodes, edges, faces) without any data mapped to them can always be useful for a number of reasons, including but not limited to understanding the mesh topology and diagnosing patterns or issues with or prior to data analysis (e.g. analyzing the mesh of a dynamical core prior to running a simulation). 




### Default `Grid` Plot


In [ ]:
uxds.uxgrid.plot().opts(width=600, height=400, title="Default Grid Plotting Method")

### Edges

We can plot the edges of each face by using the `Grid.plot.edges()` method.



In [ ]:
uxds.uxgrid.plot.edges().opts(width=600, height=400, color="Black", title="Edge Plot")

### Coordinates

UXarray stores the latitude and longitude of the corners of each face, the center of each edge, and center of each face. These coordinates can be plotted using the following methods

#### Node Coordinates


``uxgrid.plot.nodes()``

Plots the values of `node_lon` and `node_lat`

#### Edge Coordinates

``uxgrid.plot.edge_centers()``

Plots the values of `edge_lon` and `edge_lat`

#### Face Coordinates

``uxgrid.plot.face_centers()``

Plots the values of `face_lon` and `face_lat`

In [ ]:
(
    uxds.uxgrid.plot(line_color="black")
    * uxds.uxgrid.plot.nodes(marker="circle", clabel=None, size=15).relabel("Nodes")
    * uxds.uxgrid.plot.edge_centers(marker="square", clabel=None, size=15).relabel(
        "Edge Centers"
    )
    * uxds.uxgrid.plot.face_centers(marker="triangle", clabel=None, size=15).relabel(
        "Face Centers"
    )
).opts(width=600, height=400, title="Plotting Coordinates as Points")

## Visualizing  Data

If you are working with data that resides on an unstructured grid, visualization is done through a `UxDataArray`


### Default `UxDataArray` Plot

In [ ]:
uxds["random_data_face"].plot().opts(
    width=600, height=400, cmap="inferno", title="Default UxDataArray Plot"
)

### Polygons

The primary method for visualizing data in UXarray is by representing the unstructured grid as Polygons. Each face in the grid is represented internally as a polygon and rendered.


#### Vector Polygon Plots: 

``uxds['var'].plot.polygons()``

Each Polygon is rendered individually, exactly as it is mathematically represented.



#### Rasterized Polygon Plots: 

``uxds['var'].plot.rasterize(method='polygon')``

Each Polygon is approximated by rasterizing each element onto a matrix of pixels, instead of drawing each polygon individually.



In [ ]:
(
    uxds["random_data_face"]
    .plot.polygons()
    .opts(width=600, height=400, cmap="inferno", title="Vector Polygons")
    + uxds["random_data_face"]
    .plot.rasterize(method="polygon")
    .opts(width=600, height=400, cmap="inferno", title="Rasterized Polygons")
).cols(1)

### Points

Since we are able to plot the locations of our coordinates as points, we can additionally shade each point using the data that is mapped to that element.

#### Vector Point Plots

``uxds['var'].plot.points()``

Shades either the corner node, edge center, or face center coordinates with the data that is mapped to them.


In [ ]:
(
    uxds.uxgrid.plot(line_color="black")
    * uxds["random_data_node"]
    .plot.points(cmap="inferno", size=15, marker="circle", clabel=None, tools=["hover"])
    .relabel("Node Data")
    * uxds["random_data_edge"]
    .plot.points(cmap="inferno", size=15, marker="square", clabel=None, tools=["hover"])
    .relabel("Edge Data")
    * uxds["random_data_face"]
    .plot.points(
        cmap="inferno", size=15, marker="triangle", clabel=None, tools=["hover"]
    )
    .relabel("Face Data")
).opts(width=600, height=400, title="Plotting Data as Points")

## Working with Periodic Data

The dataset used in the previous examples is an extremely simple grid, consisting only of 4 hexagons. When working with a global grid that is periodic, faces that wrap around the antimeridian will cause visual artifacts unless they are corrected. The ``periodic_elements`` parameter can be used to determine how to handle these elements.
* ``periodic_elements='exclude'``: Periodic polygons are excluded from the final plot
* ``periodic_elements='split'``: Periodic polygons are split along the antimeridian
* ``periodic_elements='ignore'``: Periodic polygons are ignored

```{warning}
It is suggested to keep ``periodic_elements='exclude'`` when working with moderatly large datasets, as there is a significant overhead needed correct the antimeridian faces.
```


In [ ]:
base_path = "../../test/meshfiles/mpas/QU/"
grid_path = base_path + "oQU480.231010.nc"
uxds_mpas = ux.open_dataset(grid_path, grid_path)

In [ ]:
(
    uxds_mpas["bottomDepth"]
    .plot(cmap=ux.cmaps.sequential_blue)
    .opts(width=700, height=350, title="Default Plot (Excluding Periodic Elements)")
    + uxds_mpas["bottomDepth"]
    .plot(periodic_elements="split", cmap=ux.cmaps.sequential_blue)
    .opts(width=700, height=350, title="Include Periodic Elements (Split)")
).cols(1)

## Geographic Projections & Features

UXarray's plotting API supports Cartopy Projections and Geographic features through the use of the ``GeoViews`` library. 


In [ ]:
import cartopy.crs as ccrs
import geoviews.feature as gf

In [ ]:
uxds_mpas["bottomDepth"].plot(
    projection=ccrs.Orthographic(),
    width=700,
    height=700,
    cmap=ux.cmaps.sequential_blue,
    title="Projected Polygon Plot",
) * gf.coastline(projection=ccrs.Orthographic())

### Shifting Center Coordinates

If you are using a projection with a shifted central longitude, you must use the ``Grid.set_central_longitude()`` function instead of the ``central_longitude`` parameter in the Projection.

```{warning}
The ``Grid.set_central_longitude()`` method performs an in-place operation. If you want to revert back to the original extent, you can run ``Grid.set_central_longitude(0.0)``
```

In [ ]:
central_longitude = 180

uxds_mpas.uxgrid.set_central_longitude(central_longitude)

uxds_mpas["bottomDepth"].plot.rasterize(
    method="polygon",
    projection=ccrs.Orthographic(),
    width=700,
    height=700,
    cmap=ux.cmaps.sequential_blue,
    title="Projected Polygon Plot (Centered about 180 degrees longitude)",
) * gf.coastline(projection=ccrs.Orthographic(central_longitude=central_longitude))